In [66]:
import pandas as pd
import altair as alt
import numpy as np

# read the file
path = "/Users/leo/Documents/Abhijit Roy/U-Mich/SIADS-696-Milestone II/"
df_sentiment_raw = pd.read_csv(path+ 'final_sentiment.csv')
df_sentiment = df_sentiment_raw[df_sentiment_raw['topic_id'] != 4]

df_sentiment.head()


,Unnamed: 0,id,date_x,topic_id,topic_prob,article,sent_scores,compound,comp_score,word_count
0,0,1,2004-01-01,82,0.761872,NaN,"{'neg': 0.013, 'neu': 0.879, 'pos': 0.109, 'co...",0.9601,pos,211.0
1,1,2,2004-01-01,40,0.398252,NaN,"{'neg': 0.2, 'neu': 0.777, 'pos': 0.023, 'comp...",-0.9900,neg,196.0
2,2,3,2004-01-01,39,0.968444,NaN,"{'neg': 0.06, 'neu': 0.861, 'pos': 0.078, 'com...",0.8555,pos,332.0
3,3,4,2004-01-01,23,0.414177,NaN,"{'neg': 0.064, 'neu': 0.931, 'pos': 0.005, 'co...",-0.9337,neg,239.0
4,4,5,2004-01-01,80,0.852831,NaN,"{'neg': 0.038, 'neu': 0.915, 'pos': 0.047, 'co...",-0.5423,neg,258.0


In [67]:
df_topic = pd.read_csv(path+ 'final_topic.csv')
df_topic.head()

,Unnamed: 0,topic_id,top_terms,MainTopic,SubTopic,rep_docs
0,0,0,"['student', 'school', 'education', 'teacher', ...",Education,NaN,candidate thronged government school city appl...
1,1,1,"['police', 'suicide', 'body', 'murder', 'found...",Crime & Corrution,Violence,four policeman beaten sector police post resid...
2,2,2,"['bjp', 'party', 'candidate', 'congress', 'lea...",Politics & Election,NaN,shiv sena chief uddhav thackeray friday said b...
3,3,3,"['railway', 'train', 'passenger', 'station', '...",Logistics,NaN,day technical snag disrupted service western r...
4,4,4,"['', '', '', '', '', '', '', '', '', '']",NaN,NaN,NaN


In [68]:
df_topic.MainTopic.nunique()

18

In [69]:
df_topic.MainTopic.unique()

array(['Education', 'Crime & Corrution', 'Politics & Election',
       'Logistics', nan, 'Natural Disaster & Accidents',
       'Energy & Natural Resources', 'Entertainment & Sports',
       'Judiciery', 'WildLIfe & Planet', 'LIfeStyle', 'Healthcare',
       'Civic', 'Agriculture, Bovinae, Fishery', 'Business',
       'Travel & Tourism', 'Terrorism', 'Religion',
       'Art, Culture, & Literature'], dtype=object)

In [70]:
df_topic_time = pd.merge(df_sentiment[['date_x','topic_id']], df_topic[['topic_id','MainTopic']], how='inner', on = 'topic_id')

In [71]:
df_topic_time.head()

,date_x,topic_id,MainTopic
0,2004-01-01,82,Business
1,2004-01-01,82,Business
2,2004-01-01,82,Business
3,2004-01-02,82,Business
4,2004-01-04,82,Business


In [72]:
df_topic_time['year'] = pd.DatetimeIndex(df_topic_time['date_x']).year
df_topic_time['month'] = pd.DatetimeIndex(df_topic_time['date_x']).month
df_topic_time['quarter'] = pd.DatetimeIndex(df_topic_time['date_x']).quarter
df_topic_time['year-quarter'] = df_topic_time['year'].map(str) + '-' + df_topic_time['quarter'].map(str)
df_topic_time['cnt'] = 1

df_topic_time.head()

,date_x,topic_id,MainTopic,year,month,quarter,year-quarter,cnt
0,2004-01-01,82,Business,2004,1,1,2004-1,1
1,2004-01-01,82,Business,2004,1,1,2004-1,1
2,2004-01-01,82,Business,2004,1,1,2004-1,1
3,2004-01-02,82,Business,2004,1,1,2004-1,1
4,2004-01-04,82,Business,2004,1,1,2004-1,1


In [86]:
df_new = df_topic_time.groupby(["year", "MainTopic"], as_index=False)["cnt"].count()
df_new.head()

,year,MainTopic,cnt
0,2004,"Agriculture, Bovinae, Fishery",1643
1,2004,"Art, Culture, & Literature",202
2,2004,Business,5541
3,2004,Civic,1481
4,2004,Crime & Corrution,4368


In [74]:
df_year_topic = df_topic_time.groupby(["year"], as_index=False)["cnt"].count()

df_year_topic.head()

,year,cnt
0,2004,47650
1,2005,27430
2,2006,35643
3,2007,50063
4,2008,84179


In [88]:
df_topic = df_topic_time.groupby(["MainTopic"], as_index=False)["cnt"].count()

df_topic

,MainTopic,cnt
0,"Agriculture, Bovinae, Fishery",65542
1,"Art, Culture, & Literature",13091
2,Business,212503
3,Civic,115483
4,Crime & Corrution,274748
5,Education,182580
6,Energy & Natural Resources,85168
7,Entertainment & Sports,208537
8,Healthcare,180027
9,Judiciery,142215


In [75]:
df_Report1 = pd.merge(
    df_new, 
    df_year_topic[['year','cnt']], 
    how='inner', 
    on = 'year')

df_Report1["cnt_normalized"] = df_Report1["cnt_x"]/df_Report1["cnt_y"]

df_Report1.head()

,year,MainTopic,cnt_x,cnt_y,cnt_normalized
0,2004,"Agriculture, Bovinae, Fishery",1643,47650,0.034481
1,2004,"Art, Culture, & Literature",202,47650,0.004239
2,2004,Business,5541,47650,0.116285
3,2004,Civic,1481,47650,0.031081
4,2004,Crime & Corrution,4368,47650,0.091668


In [76]:


source = df_Report1

alt.Chart(source).mark_area().encode(
    #alt.X('year:T',
    #    axis=alt.Axis(format='%Y', domain=True, tickSize=0)
    #),
    alt.X('year:N'),
    alt.Y('sum(cnt_normalized):Q', stack='center', axis=None),
    alt.Color('MainTopic:N', scale=alt.Scale(scheme='category20b')
    )
).properties(
    width=700,
    height=500
)

alt.Chart(...)

In [77]:
#topic = 'Politics & Election'
#topic = 'Crime & Corrution'
#topic = 'Healthcare'
#topic = 'Education'
#topic = 'Religion'
#topic = 'Business'
#topic = 'Agriculture, Bovinae, Fishery'
#topic = 'Civic'
#topic = 'Judiciery'
#topic = 'Logistics'
#topic = 'Entertainment & Sports'
#topic = 'LifeStyle'
topic = 'Terrorism'

df_topic_selected = df_topic[df_topic['MainTopic'] == topic]

df_topic_sentiment_time = pd.merge(
    df_sentiment[['date_x','topic_id','compound','comp_score']], 
    df_topic_selected[['topic_id','MainTopic']], 
    how='inner', on = 'topic_id')

df_topic_sentiment_time['year'] = pd.DatetimeIndex(df_topic_sentiment_time['date_x']).year
df_topic_sentiment_time['month'] = pd.DatetimeIndex(df_topic_sentiment_time['date_x']).month
#df_topic_time['quarter'] = pd.DatetimeIndex(df_topic_time['date_x']).quarter
df_topic_sentiment_time['year-month'] = df_topic_sentiment_time['year'].map(str) + '-' + df_topic_sentiment_time['month'].map(str)

df_topic_sentiment_time['pos'] = np.where(df_topic_sentiment_time['comp_score']== 'pos', 1, 0)
df_topic_sentiment_time['neg'] = np.where(df_topic_sentiment_time['comp_score']== 'neg', -1, 0)


df_topic_sentiment_time.head()

,date_x,topic_id,compound,comp_score,MainTopic,year,month,year-month,pos,neg
0,2004-01-01,39,0.8555,pos,Terrorism,2004,1,2004-1,1,0
1,2004-01-02,39,-0.1611,neg,Terrorism,2004,1,2004-1,0,-1
2,2004-01-04,39,-0.7689,neg,Terrorism,2004,1,2004-1,0,-1
3,2004-01-04,39,-0.9648,neg,Terrorism,2004,1,2004-1,0,-1
4,2004-01-07,39,-0.7351,neg,Terrorism,2004,1,2004-1,0,-1


In [78]:
df_topic_sentiment_time_pos = df_topic_sentiment_time.groupby(["year-month",], as_index=False)["pos","neg"].sum()

df_topic_sentiment_time_pos.head()

,year-month,pos,neg
0,2004-1,40,-84
1,2004-10,26,-75
2,2004-11,35,-64
3,2004-12,21,-43
4,2004-2,29,-63


In [79]:
source = df_topic_sentiment_time_pos

pos = alt.Chart(source).mark_bar().encode(
    x="year-month:T",
    y="pos:Q",
    color=alt.condition(
        alt.datum.nonfarm_change > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("green")  # The negative color
    )
).properties(width=800)

neg = alt.Chart(source).mark_bar().encode(
    x="year-month:T",
    #y="pos:Q",
    y= "neg:Q",
    color=alt.condition(
        alt.datum.nonfarm_change < 0,
        alt.value("steelblue"),  # The positive color
        alt.value("red")  # The negative color
    )
).properties(width=800)

pos + neg

alt.LayerChart(...)

In [85]:
#topic = 'Politics & Election'
#topic = 'Crime & Corrution'
#topic = 'Healthcare'
#topic = 'Education'
#topic = 'Religion'
#topic = 'Business'
#topic = 'Agriculture, Bovinae, Fishery'
#topic = 'Civic'
#topic = 'Judiciery'
#topic = 'Logistics'
#topic = 'Entertainment & Sports'
#topic = 'LifeStyle'
#topic = 'Terrorism'

#df_topic_selected = df_topic[df_topic['MainTopic'] == topic]

df_topic_selected = df_topic

df_topic_sentiment_time = pd.merge(
    df_sentiment[['date_x','topic_id','compound','comp_score']], 
    df_topic_selected[['topic_id','MainTopic']], 
    how='inner', on = 'topic_id')

df_topic_sentiment_time['year'] = pd.DatetimeIndex(df_topic_sentiment_time['date_x']).year
df_topic_sentiment_time['month'] = pd.DatetimeIndex(df_topic_sentiment_time['date_x']).month
#df_topic_time['quarter'] = pd.DatetimeIndex(df_topic_time['date_x']).quarter
df_topic_sentiment_time['year-month'] = df_topic_sentiment_time['year'].map(str) + '-' + df_topic_sentiment_time['month'].map(str)


df_topic_sentiment_time['comp_score_new'] = np.where(
    df_topic_sentiment_time['compound'] > 0.05, 'pos', np.where(
    df_topic_sentiment_time['compound'] < -0.05, 'neg', 'neu')) 

df_topic_sentiment_time['pos'] = np.where(df_topic_sentiment_time['comp_score_new']== 'pos', 1, 0)
df_topic_sentiment_time['neg'] = np.where(df_topic_sentiment_time['comp_score_new']== 'neg', 1, 0)
df_topic_sentiment_time['neu'] = np.where(df_topic_sentiment_time['comp_score_new']== 'neu', 1, 0)

df_topic_sentiment_time_pos = df_topic_sentiment_time.groupby(["year",], as_index=False)["pos","neg","neu"].sum()


df_topic_sentiment_time['count'] = 1
#df_topic_sentiment_time_pos.head()

#df_topic_sentiment_time[['year','MainTopic','comp_score_new']]

df = df_topic_sentiment_time.groupby(["year","comp_score_new"], as_index=False)["count"].count()

#df1 = df[['year','comp_score_new','count']]
df

#df_topic_sentiment_time.head()

,year,comp_score_new,count
0,2004,neg,19414
1,2004,neu,762
2,2004,pos,27474
3,2005,neg,12473
4,2005,neu,364
5,2005,pos,14593
6,2006,neg,16981
7,2006,neu,491
8,2006,pos,18171
9,2007,neg,21899


In [65]:
source_neg = df1

neg = alt.Chart(source_neg).mark_line().encode(
    x='year:O',
    y='count:Q',
    color= 'comp_score_new:N'
    
    
)




neg

alt.Chart(...)